# Digital Speech Processing
### Prof. Dr. Rodrigo Guido

### Student: Álvaro Leandro Cavalcante Carneiro

# Short Test Challenge
Creating and normalizing a filter based on new formula.

## Importing the necessary libraries

In [32]:
import math
import numpy as np
import random

## Defining filter parameters
First of all, it's important to define the necessary parameters, like the upper and lower bound of accepted Gibbs effect and the frequency.

In [2]:
upper_bound_freq = 0.45
lower_bound_freq = 0.15

passband_upper = 1.01
passband_lower = 0.99

stopband_upper = 0.06
stopband_lower = 0

The cutoff frequency here is just the different between the upper bound and lower bound. 

In [3]:
cutoff = upper_bound_freq - lower_bound_freq

After that, it's important to get the lower value of Gibbs effect to get the dB and choose the correct window.

In [4]:
passband_range = passband_upper - passband_lower
stopband_range = stopband_upper - stopband_lower

min_frequency = min([passband_range, stopband_range])

Now, it's possible to calculate the dB value and choose the window that we'll use in our equation.

In [5]:
dB_to_be_filtered = 20* math.log(min_frequency, 10)

windows = {'rectangular': -21, 'barlett': -25, 'hanning': -44, 'hamming': -53, 'blackman': -74}
used_window = ''

for window in windows:
    if windows[window] < dB_to_be_filtered:
        used_window = window
        print('Window to be used:', window)
        break

Window to be used hanning


We can also get the order, using the function defined by hanning window.

In [9]:
order = int(np.ceil(3.1 / ((cutoff * math.pi) / (2 * math.pi))))
print('Filter order is:', order, 'Which will generate:', order+1, 'coefficients.')

Filter order is: 21 Which will generate: 22 coefficients.


## Function to get low pass filter
The function bellow will calculate the low pass, using de SINC function considering all the parameters we found until now.

In [12]:
def get_low_pass_filter(order, cutoff):
    final_filter = []
    for n in range(order + 1):
        first_part = math.sin( (cutoff * math.pi)  * (n - (order/2)) )
        second_part = (math.pi * (n - (order/2)))
        if second_part == 0:
            print('Zero division')
            value = 1
        else:
            value = first_part / second_part
        final_filter.append(value)
    
    return final_filter

In [15]:
freq_filter = get_low_pass_filter(order, cutoff)
print(sum(freq_filter))
freq_filter

1.0412224393462681


[-0.013762825171487384,
 0.015211543610591289,
 0.03698717215057592,
 0.030010543871903557,
 -0.007660713348027363,
 -0.05156657914607532,
 -0.06302581895631432,
 -0.014227039074908047,
 0.09003163161571058,
 0.20959397551993025,
 0.2890193286012348,
 0.2890193286012348,
 0.20959397551993025,
 0.09003163161571058,
 -0.014227039074908047,
 -0.06302581895631432,
 -0.05156657914607532,
 -0.007660713348027363,
 0.030010543871903557,
 0.03698717215057592,
 0.015211543610591289,
 -0.013762825171487384]

## Hanning window
Above we can see the 22 filter coefficients. Also, the sum of them is 1.04, so it's necessary to normalize the values, but let's apply the hanning window before.

In [22]:
def get_window_value(order):
    window = []
    for n in range(order + 1):
        value = 0.5 - (0.5* math.cos(2*math.pi*(n/ order)))
        window.append(value)
    return window

In [25]:
han_window = get_window_value(order)
print(len(han_window))
han_window

22


[0.0,
 0.022213597106929606,
 0.08688061284200255,
 0.1882550990706332,
 0.31732948781680237,
 0.4626349532067878,
 0.6112604669781572,
 0.7499999999999999,
 0.8665259359149131,
 0.9504844339512095,
 0.9944154131125642,
 0.9944154131125642,
 0.9504844339512095,
 0.8665259359149131,
 0.7500000000000002,
 0.6112604669781573,
 0.46263495320678827,
 0.3173294878168027,
 0.1882550990706333,
 0.08688061284200255,
 0.022213597106929717,
 0.0]

As saw, the window has the same len as the filter. Now, it's possibel to apply the multiplication.

In [27]:
window_filter = np.array(han_window) * np.array(freq_filter)
window_filter

array([-0.        ,  0.0003379 ,  0.00321347,  0.00564964, -0.00243097,
       -0.0238565 , -0.03852519, -0.01067028,  0.07801474,  0.19921581,
        0.28740528,  0.28740528,  0.19921581,  0.07801474, -0.01067028,
       -0.03852519, -0.0238565 , -0.00243097,  0.00564964,  0.00321347,
        0.0003379 , -0.        ])

## Normalizing results
The function above normalize the filter results

In [29]:
def normalized_coef(coef):
    print('Sum befone normalization', sum(coef))
    normalized = [x/sum(coef) for x in coef]
    print('Sum after normalization', sum(normalized))
    return normalized

In [30]:
filter_normalized = normalized_coef(window_filter)
filter_normalized

Sum befone normalization 0.9967077925322666
Sum after normalization 1.0000000000000002


[-0.0,
 0.0003390192227570301,
 0.0032240825323241917,
 0.00566829912648234,
 -0.002438999936846753,
 -0.02393530190996913,
 -0.03865244338969116,
 -0.01070552411261057,
 0.0782724329359798,
 0.1998738373214789,
 0.2883545982100954,
 0.2883545982100954,
 0.1998738373214789,
 0.0782724329359798,
 -0.010705524112610575,
 -0.03865244338969117,
 -0.023935301909969155,
 -0.0024389999368467554,
 0.005668299126482344,
 0.0032240825323241917,
 0.00033901922275703176,
 -0.0]

## Difference equation
Considering a random input filter x, let's apply the difference equation with our filter coefficients.

In [54]:
x = []
for _ in range(110):
    x.append(random.random())

print(x)

[0.564343636035832, 0.5103994774565198, 0.5519183935166486, 0.3013805927850619, 0.951069029196288, 0.48687773004162116, 0.9366909564640711, 0.8022964505006122, 0.017972734095814014, 0.5824443182587522, 0.7598062101149874, 0.7741839720691577, 0.24594260580252414, 0.43386536336805803, 0.2501976330244109, 0.6270033751469661, 0.9975115465488796, 0.477515779420414, 0.1414634081944378, 0.8887747126394219, 0.6485570377684501, 0.38477326234400466, 0.250306815824234, 0.9893369807909844, 0.7273523419984347, 0.47035288736709124, 0.6525839708541044, 0.7513971597558528, 0.8996022649093708, 0.6578190325138621, 0.7537777190818044, 0.21918281541877127, 0.8722930189937131, 0.2496763577479577, 0.5949970463219577, 0.03946578234517395, 0.6171631384844224, 0.07767565241162921, 0.7990650054349588, 0.2008664887733036, 0.12003930674114627, 0.16087530517809134, 0.5107377654345802, 0.7565515095762108, 0.5156432931482229, 0.13952822891347394, 0.9369298555583007, 0.7134888842907944, 0.12319245545365043, 0.7839717

We can check above the filter to be filtered, using just 100 samples in this case. After that, we just apply a piecewise multiplication in the x input signal by the filter coefficients.

In [55]:
y = []

for n in range(0, len(x), order+1):
    input_sample = x[n: n + order+1][::-1]
    y[n: n + order] = np.multiply(filter_normalized, input_sample)

In [61]:
y

[-0.0,
 0.00021987330285786174,
 0.002865483026192213,
 0.0008018569130977466,
 -0.0011646609558497178,
 -0.02387574002532766,
 -0.0242352124630134,
 -0.002678496793260921,
 0.03395969755747083,
 0.04915749238259432,
 0.2232395082066977,
 0.21909361443524253,
 0.11641538091646952,
 0.0014067696241908,
 -0.00858900399629618,
 -0.036205394168363186,
 -0.011653565461786662,
 -0.0023196573021466516,
 0.0017083153508222974,
 0.0017794304518054564,
 0.0001730352341429045,
 -0.0,
 -0.0,
 0.00017314992027029372,
 0.000518675261307008,
 0.0006804186975443851,
 -0.0004899133534327165,
 -0.019125862150776864,
 -0.003002353757597826,
 -0.0066070548604594,
 0.0030890828018786034,
 0.11892434284331542,
 0.07199532582097239,
 0.25152970301340327,
 0.043808910392675225,
 0.05900001596546635,
 -0.00704229751431131,
 -0.03477182561764741,
 -0.01798491787304966,
 -0.0015916522637003654,
 0.0026661008606013313,
 0.002345043980682245,
 0.000335404254272548,
 -0.0,
 -0.0,
 0.00012286788667755007,
 0.0005440